In [3]:
#Importing the libraries
import matplotlib.pyplot as plt
%matplotlib inline

## DnaA boxes

DnaA protein are binded to regions called dnaA boxes in the region knowed as oric, where the replication processes begin. We will start with a bacterium called Vibrio cholerae, and then design a computational approach for finding ori in other bacteria genomes.
Here is the nucleotide sequence appearing in the ori of Vibrio cholerae: